<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/stability%20and%20variance%20analyzer%20of%20one%20football%20player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import depth_pro
model, transform = depth_pro.create_model_and_transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
model = model.half().to(device).eval()
from pickle import FRAME
from google.colab.patches import cv2_imshow
from decord import VideoReader
import mediapipe as mp
import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics

video_path = "kid1.mp4"
vr = VideoReader(video_path)
frame_count = len(vr)
print(f"Total frames: {frame_count}")
output_path = "kid1_fin.mp4"

def approximate_intrinsics(width, height):
    c_x, c_y = width/2, height/2
    return float(c_x), float(c_y)


def pixel_to_camera_coords(x, y, Z, f_x, f_y, c_x, c_y):
   X = (x - c_x) * Z / f_x
   Y = (y - c_y) * Z / f_y
   return X, Y, Z

def stability_calc(frame, depth_map, fx,fy, results, c_x, c_y):
  points = {}
  mp_pose = mp.solutions.pose

  h, w, _ = frame.shape
  for id, lm in enumerate(results.pose_landmarks.landmark):
        cx, cy = int(lm.x * w), int(lm.y * h)

        cz = depth_map[cy, cx]
        X, Y, Z = pixel_to_camera_coords(cx, cy, cz, fx, fy, c_x, c_y)
        if id == mp_pose.PoseLandmark.LEFT_ANKLE.value:
            points["left_ankle"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.RIGHT_ANKLE.value:
            points["right_ankle"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.LEFT_HIP.value:
            points["left_hip"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.RIGHT_HIP.value:
            points["right_hip"] = (X, Y, Z)

        if id in [mp_pose.PoseLandmark.LEFT_ANKLE.value,
                  mp_pose.PoseLandmark.RIGHT_ANKLE.value]:
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

  mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


  com_x = (points["left_hip"][0] + points["right_hip"][0]) / 2
  com_y = (points["left_hip"][1] + points["right_hip"][1]) / 2
  com_z = (points["left_hip"][2] + points["right_hip"][2]) / 2

  foot_midx = (points["left_ankle"][0] + points["right_ankle"][0]) / 2
  foot_midy = (points["left_ankle"][1] + points["right_ankle"][1]) / 2
  foot_midz = (points["left_ankle"][2] + points["right_ankle"][2]) / 2

  stance_width= abs(points["left_ankle"][0] -  points["right_ankle"][0])
  sway = abs(com_x - foot_midx)
  norm= sway/stance_width
  return norm

# Sample every 5 seconds (assuming 30 fps)
fps = 7
step = fps * 1
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_draw = mp.solutions.drawing_utils

sample_frame = vr[0].asnumpy()
height, width, _ = sample_frame.shape
c_x,c_y= approximate_intrinsics(width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
var = 0
norms = []
for i in range(0, frame_count, step):

    frame = vr[i].asnumpy()
    rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    img = transform(rgb).unsqueeze(0).to(device).half()
    with torch.no_grad():
        pred = model.infer(img)
        depth_map = pred["depth"].squeeze().cpu().numpy()
        depth_map = cv2.resize(depth_map, (frame.shape[1], frame.shape[0]))
        intrinsics = pred["focallength_px"].cpu().numpy()
        if np.ndim(intrinsics) == 0:
          fx = fy = float(intrinsics)
        else:
          fx, fy = intrinsics

    if results.pose_landmarks:
      norm= stability_calc(frame, depth_map, fx,fy, results,c_x,c_y)
      norms.append(norm)
    if len(norms) > 1:
      var = statistics.variance(norms)
    print("stability is" , norm)
    cv2.putText(frame,
            f"Frame: {i} | Stability: {norm:.2f} | Variance: {var:.2f}",
            (10, 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 200),
            2,
            cv2.LINE_AA)
    out.write(frame)
out.release()
print(f"Video saved to {output_path}")



Total frames: 223
stability is 0.004023164
stability is 0.29592225
stability is 0.19619568
stability is 0.39001125
stability is 0.016216435
stability is 0.48599654
stability is 0.041316293
stability is 0.64609855
stability is 0.008657447
stability is 0.65711224
stability is 0.0075008287
stability is 1.5386063
stability is 0.016657379
stability is 0.21284124
stability is 0.2187201
stability is 0.32547885
stability is 0.17471206
stability is 0.19856887
stability is 0.19902043
stability is 0.22635767
stability is 0.34788218
stability is 0.023908313
stability is 0.75669277
stability is 0.021261385
stability is 0.1478696
stability is 0.076974
stability is 0.07244681
stability is 0.63067746
stability is 0.008173687
stability is 0.67768776
stability is 0.2723846
stability is 0.10753688
Video saved to kid1_fin.mp4
